In [54]:
#Get classifications downloaded from s3
#!s3cmd s3://dse-pjavaji/Topics ./topics

In [55]:
#!cat ./topics/Topics/Topic_10_12.json/part-00000 | head

In [56]:
!cd ~/documents/DSE/dhofman/capstone

In [57]:
#when a topic is defined for a tweet
# query_prefix = """WITH {json} as data """
# query="""UNWIND data.topics as q \
# MATCH (t:Tweet_{0} {{id_str:q.tweetid}}) \
# SET t.topic = q.topic"""

query="""MATCH (t:Tweet_{0}) \
WHERE t.id_str IN {1} \
SET t.topic = "{2}" \
RETURN count(t)"""


In [58]:
partitions = ['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15',\
              '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', \
              '2016_2_2_1', '2016_2_2_2', '2016_2_2_3', '2016_2_3', \
              '2016_2_4_1', '2016_2_4_2', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9',\
              '2016_2_10', '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', \
              '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_2_27', \
              '2016_3_1_1', '2016_3_1_2','2016_3_1_3',\
              '2016_3_2_1','2016_3_2_2','2016_3_2_3','2016_3_3_1', '2016_3_3_2', '2016_3_3_3', \
             '2016_3_4_1', '2016_3_4_2','2016_3_4_3', '2016_3_5_1','2016_3_5_2','2016_3_5_3','2016_3_5_4',\
              '2016_3_6_1', '2016_3_6_2','2016_3_6_3','2016_3_6_4',\
              '2016_3_7_1','2016_3_7_2','2016_3_7_3','2016_3_7_4', \
              '2016_3_8_1','2016_3_8_2','2016_3_8_3','2016_3_8_4', \
              '2016_3_9_1','2016_3_9_2','2016_3_9_3','2016_3_9_4',\
             '2016_3_10_1','2016_3_10_2','2016_3_10_3','2016_3_10_4', '2016_3_11']


In [59]:
prequery = """CREATE INDEX ON :Tweet_{0}(id_str)"""

In [60]:
print partitions

['2015_10_11', '2015_10_12', '2015_10_13', '2015_10_14', '2015_10_15', '2016_1_26', '2016_1_27', '2016_1_28', '2016_1_29', '2016_1_30', '2016_2_2_1', '2016_2_2_2', '2016_2_2_3', '2016_2_3', '2016_2_4_1', '2016_2_4_2', '2016_2_5', '2016_2_6', '2016_2_7', '2016_2_8', '2016_2_9', '2016_2_10', '2016_2_11', '2016_2_12', '2016_2_13', '2016_2_14', '2016_2_15', '2016_2_23', '2016_2_24', '2016_2_25', '2016_2_26', '2016_2_27', '2016_3_1_1', '2016_3_1_2', '2016_3_1_3', '2016_3_2_1', '2016_3_2_2', '2016_3_2_3', '2016_3_3_1', '2016_3_3_2', '2016_3_3_3', '2016_3_4_1', '2016_3_4_2', '2016_3_4_3', '2016_3_5_1', '2016_3_5_2', '2016_3_5_3', '2016_3_5_4', '2016_3_6_1', '2016_3_6_2', '2016_3_6_3', '2016_3_6_4', '2016_3_7_1', '2016_3_7_2', '2016_3_7_3', '2016_3_7_4', '2016_3_8_1', '2016_3_8_2', '2016_3_8_3', '2016_3_8_4', '2016_3_9_1', '2016_3_9_2', '2016_3_9_3', '2016_3_9_4', '2016_3_10_1', '2016_3_10_2', '2016_3_10_3', '2016_3_10_4', '2016_3_11']


In [61]:
from neo4j.v1 import GraphDatabase, basic_auth
import time
import os
import sys
import ast
import json
import logging

class neo4j_writer:
    
    def __init__(self,drivername,logger):
        self.driver = None
        if (drivername=='remote'):
            self.remotedriver()
        else:
            self.localdriver()
            
        self.logger = logger
        
    def localdriver(self):
        if (self.driver == None):
            self.driver = GraphDatabase.driver("bolt://127.0.0.1", auth=basic_auth("neo4j", "lajolla"))
    
    def remotedriver(self):
        if (self.driver == None):
            self.driver = GraphDatabase.driver("bolt://sauce5.sdsc.edu", auth=basic_auth("neo4j", "lajolla"))
    
    def runcmd(self,query):
        
        session = self.driver.session()
        
        try:
            session.run(query)
        except:
            self.logger.info('Error - {0}'.format(query))
        finally:
            session.close()
        
    def runcmd_result(self,query):
        
        startsession = time.time()
        session = self.driver.session()
        #print time.time()-startsession, session
        #sys.stdout.flush()
        count=0
        
        try:
            startqry = time.time()
            result = session.run(query)
#             print 'result start - ', time.time()-startqry
#             sys.stdout.flush()
            
            for record in result:
                count=record[0]

#             print 'result ends - ', time.time()-startqry 
#             sys.stdout.flush()
            
        except Exception as e:
            self.logger.info('Error - {0} {1}'.format(e, query))
        finally:
            session.close()
            
        return count

class topicreader:

    def __init__(self,partition,logger):
        self.partition=partition
        self.topicfolder()
        self.logger = logger
        
    def topicfolder(self):
        fnprefix = './topics/Topics/Topic_'
        ddmm             = self.partition.split('_')
        topicfolder      = fnprefix + '_'.join(ddmm[1:(len(ddmm))])+'.json'
        self.topicfolder = topicfolder
    
    def topicfiles(self):      
        return [(self.topicfolder +'/'+afile) \
                 for afile in os.listdir(self.topicfolder) \
                 if (afile != '_SUCCESS')]
        
    def formatline(self,line):
        line2 = line[1:-3]
        line2 = line2.split('},')
        return line2
        
    def formatitem(self,item):
        item   = item + '}'
        item   = item.lstrip(' ')
        d_item = ast.literal_eval(item)
        return d_item
    
    def getitems(self):
        for filename in self.topicfiles():
            self.logger.info('reading file {0}'.format(filename))
            sys.stdout.flush()
            with open(filename, 'rb') as f:
                for line in f: 
                    self.logger.info('read line in file {0}'.format(filename))
                    sys.stdout.flush()
                    for item in self.formatline(line):
                        formatted_item = self.formatitem(item)
                        if (formatted_item['topic']!='none'):
                            yield formatted_item

class databatch:
    
    def __init__(self,query,writer,filenamepart,logger):
        self.data = []
        self.query = query
        self.writer = writer
        self.makepartitionlabel(filenamepart)
        self.requestcount = 0
        self.resultcount = 0
        self.logger = logger
        
    def add(self, item):
        self.data.append(item)
        
    def makepartitionlabel(self,filenamepart):
        ddmm = filenamepart.split('_')
        self.partition = '_'.join(ddmm[0:3])
        
    def items(self):
        return self.data
    
    def clear(self):
        self.data = []

    def startcount(self):
        self.requestcount = 0
        self.resultcount = 0
    
    #sort by tweetid to optimize performance of updates
    def sorteddata(self):
        return sorted(self.data, key=lambda k: k['tweetid'])
    
    def topicdata(self):
        
        topics = set([item['topic'] for item in self.data])
        
        topicoutput = {}
        
        for topic in topics:
            topicoutput[topic] = sorted(list([str((item['tweetid'])).encode('ascii') \
                                       for item in self.data if (item['topic']==topic)]))
            
        return topicoutput
    
        
    def processbatch(self):

        try:
            topictweets = self.topicdata()
            
            for k,v in topictweets.iteritems():
                ids = list(set(v))
                query_formatted = self.query.format(self.partition, ids, k)
#                 print 'processing topic -',k
#                 print 'for ids -', ids
#                 sys.stdout.flush()
#                 print query_formatted
                
                if (len(ids)>0):
                    count = self.writer.runcmd_result(query_formatted)
                    self.requestcount = self.requestcount + len(ids)
                    self.resultcount = self.resultcount + count
            
        except Exception as e:
            self.logger.info('Error writing batch {0}'.format(e))
            
        self.clear()


#set up logging
logger = logging.getLogger('TagTweetsByTopic')
hdlr = logging.FileHandler('./tagging_local.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hdlr.setFormatter(formatter)
logger.addHandler(hdlr) 
logger.setLevel(logging.INFO)


batchsize = 500
writer = neo4j_writer('local',logger)

#for partition in partitions:
for partition in ['2016_2_9']:
    
    prequery_formatted = prequery.format(partition)
    
    reader = topicreader(partition,logger)
    
    #add index for this partition
    #writer.runcmd(prequery_formatted)

    start = time.time() 
    data = databatch(query,writer,partition,logger)
    
    for d_item in reader.getitems():
        
        data.add(d_item)

        if (len(data.items()) >= batchsize):
            data.processbatch()
            logger.info('processing time after batch: {0}, request count: {1}, result count: {2}'.format(time.time()-start, \
            data.requestcount, data.resultcount))
            
                          
    if (len(data.items()) > 0):
        data.processbatch()
        logger.info('processing time after batch: {0}, request count: {1}, result count: {2}'.format(time.time()-start, \
            data.requestcount, data.resultcount))
                    
    
    print time.time() - start


        

INFO:TagTweetsByTopic:reading file ./topics/Topics/Topic_2_9.json/part-00000
INFO:TagTweetsByTopic:read line in file ./topics/Topics/Topic_2_9.json/part-00000
INFO:TagTweetsByTopic:processing time after batch: 1.14157104492, request count: 477, result count: 456
INFO:TagTweetsByTopic:processing time after batch: 1.86985182762, request count: 918, result count: 881
INFO:TagTweetsByTopic:processing time after batch: 2.53323888779, request count: 1368, result count: 1308
INFO:TagTweetsByTopic:processing time after batch: 3.39757490158, request count: 1834, result count: 1751
INFO:TagTweetsByTopic:processing time after batch: 4.79472184181, request count: 2298, result count: 2197
INFO:TagTweetsByTopic:processing time after batch: 5.88322997093, request count: 2761, result count: 2635
INFO:TagTweetsByTopic:processing time after batch: 6.76861596107, request count: 3200, result count: 3050
INFO:TagTweetsByTopic:processing time after batch: 7.53210496902, request count: 3665, result count: 34

174.281098843
